In [1]:
import pandas as pd

In [5]:
#read parquet file
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

In [10]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [11]:
print(len(df.columns))

19


In [14]:
# Convert pickup and dropoff times to datetime type
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [15]:
# Calculate the duration and convert to minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [17]:
january_data = df[df['tpep_pickup_datetime'].dt.month == 1]

In [18]:
january_duration_std = january_data['duration'].std()

In [19]:
#print january_duration_std
print(january_duration_std)

42.58564176425905


In [20]:
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

In [21]:
fraction_left = len(filtered_df) / len(df)

In [22]:
print(fraction_left)

0.9812202822125979


In [23]:
data = filtered_df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

In [25]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
dv.fit(data)

DictVectorizer()

In [26]:
feature_matrix = dv.transform(data)

In [29]:
num_columns = feature_matrix.shape[1]
print(num_columns)


515


In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Train a linear regression model
lr = LinearRegression()
lr.fit(feature_matrix, filtered_df['duration'])

# Make predictions on the training data
train_predictions = lr.predict(feature_matrix)

# Calculate the RMSE on the training data
train_rmse = mean_squared_error(filtered_df['duration'], train_predictions, squared=False)

/home/mrblack/anaconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [33]:
print(train_rmse)

7.649262444548727


In [34]:
validation_df = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')


In [36]:
# Convert pickup and dropoff times to datetime type
validation_df['tpep_pickup_datetime'] = pd.to_datetime(validation_df['tpep_pickup_datetime'])
validation_df['tpep_dropoff_datetime'] = pd.to_datetime(validation_df['tpep_dropoff_datetime'])

# Calculate the duration and convert to minutes
validation_df['duration'] = (validation_df['tpep_dropoff_datetime'] - validation_df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [37]:
filtered_validation_df = validation_df[(validation_df['duration'] >= 1) & (validation_df['duration'] <= 60)]

validation_data = filtered_validation_df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
validation_feature_matrix = dv.transform(validation_data)

validation_predictions = lr.predict(validation_feature_matrix)

validation_rmse = mean_squared_error(filtered_validation_df['duration'], validation_predictions, squared=False)

/home/mrblack/anaconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [38]:
print(validation_rmse)

7.811812767485678
